# Sentiment Classification

In [36]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [37]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding,LSTM
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import TimeDistributed,Flatten

from tensorflow.keras.datasets import imdb

In [38]:
num_words = 10000

In [39]:
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=num_words)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [40]:
maxlen = 300

In [41]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

Number of review, number of words in each review

In [42]:
print("train_data ", x_train.shape)
print("_"*50)
print("test_data ", x_test.shape)
print("_"*50)

train_data  (25000, 300)
__________________________________________________
test_data  (25000, 300)
__________________________________________________


In [43]:
print("Maximum value of a word index ")
print(max([max(sequence) for sequence in x_train]))
print("Maximum length num words of review in train ")
print(max([len(sequence) for sequence in x_train]))

Maximum value of a word index 
9999
Maximum length num words of review in train 
300


Number of labels

In [44]:
print("train_labels ", y_train.shape)
print("_"*25)
print("\ntest_labels ", y_test.shape)
print("_"*25)

train_labels  (25000,)
_________________________

test_labels  (25000,)
_________________________


Feature value

In [45]:
print("--Movie Review--\n")
print(x_train[0])


--Movie Review--

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    1   14
   22   16   43  530  973 1622 1385   65  458 4468   66 3941    4  173
   36  256    5   25  100   43  838  112   50  670    2    9   35  480
  284    5  150    4  172  112  167    2  336  385   39    4  172 4536
 1111   17  546   38   13  447    4  192   50   16    6  147 2025   19
   14   22    4 1920 4613  469    4   22   71   87   12   16   43  530
   38   76   15   13 1247    4   22   17  515   17   12   16  626   18
    2    5   62  386   12    8  316    8  106    5    4 2223 5244   16
  480   66 3785   33    4  130   12   16   38  619    5   2

Label value

In [46]:
print("--Sentiment--> ",format(y_train[0]))

--Sentiment-->  1


First, retrieve a dictionary that contains mapping of words to their index in the IMDB dataset

In [47]:
print("-— Decoded Movie Review -— ")
index = tf.keras.datasets.imdb.get_word_index()

-— Decoded Movie Review -— 


Now use the dictionary to get the original words from the encodings, for a particular sentence

In [48]:
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in x_train[0] if i != 0] )
print(decoded)



# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

Get the sentiment for the above sentence
- positive (1)
- negative (0)

In [49]:
if(y_train[0] == 0):
  print("Negative Sentiment : 0")
elif(y_train[0] == 1):
  print("Positive Sentiment : 1")

Positive Sentiment : 1


In [50]:
model = Sequential()

In [51]:
embed_dim = 100
vocabSize=num_words

model.add(Embedding(vocabSize, embed_dim,input_length = maxlen))

In [52]:
model.add(LSTM(units=16, return_sequences=True))

In [53]:
# Add timeDistributed Layer
model.add(TimeDistributed(Dense(100, activation="relu")))

In [54]:
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [55]:
optimizer = Adam(lr=1e-3)

In [56]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [57]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 100)          1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 300, 16)           7488      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 300, 100)          1700      
_________________________________________________________________
flatten_1 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 30001     
Total params: 1,039,189
Trainable params: 1,039,189
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.fit(x_train, y_train,
          validation_split=0.05, epochs=5, batch_size=32)

Epoch 1/5
743/743 [==============================] - 18s 22ms/step - loss: 0.4305 - accuracy: 0.7692 - val_loss: 0.2577 - val_accuracy: 0.8992
Epoch 2/5
743/743 [==============================] - 16s 21ms/step - loss: 0.1656 - accuracy: 0.9375 - val_loss: 0.2930 - val_accuracy: 0.8824
Epoch 3/5
743/743 [==============================] - 16s 22ms/step - loss: 0.0852 - accuracy: 0.9686 - val_loss: 0.3714 - val_accuracy: 0.8832
Epoch 4/5
743/743 [==============================] - 16s 21ms/step - loss: 0.0567 - accuracy: 0.9803 - val_loss: 0.5401 - val_accuracy: 0.8800
Epoch 5/5
743/743 [==============================] - 16s 21ms/step - loss: 0.0267 - accuracy: 0.9917 - val_loss: 0.6312 - val_accuracy: 0.8872


In [59]:
result = model.evaluate(x_test, y_test)

782/782 [==============================] - 6s 7ms/step - loss: 0.6539 - accuracy: 0.8723


In [60]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 87.23%


In [61]:
print(x_test.shape)
sentiment = model.predict(x_test)
print(max(sentiment))
y_test[2]

(25000, 300)
[1.]


1

In [62]:
i = 10 # Some test sentence sample
p = model.predict(np.array([x_test[i]])) # Predict on it
sentiment = p[0][0]
print("sentiment ",sentiment)


sentiment  0.9999999


In [63]:
print("Actual Sentiment: ",y_test[10])

if(sentiment>0.5):
  print("\nPredicted Sentiment: 1")
elif(sentiment<=0.5):
  print("\nPredicted Sentiment: 0")



Actual Sentiment:  1

Predicted Sentiment: 1
